# Notebook zum Topic Modeling der Drittmitteldaten

> Dieses Notebook folgt dem gleichen Aufbau wie das _Topic Modeling_ für die Publikationsdaten.

Gleich am Anfang ist einzustellen, ob man einen _Grid Search_ durchführen möchte oder eine _Evaluation Pipeline_. Erstere sucht über die systematische Veränderung von Modellparametern das beste Modell; Zweitere basiert auf den Ergebnissen des _Grid Search_ und kann deswegen nur ausgeführt werden, wenn dieser zumindest einmal gelaufen ist. Die Ergebnisse des _Grid Search_ werden dafür geladen.

Das Notebook folgt einem bestimmten Aufbau:
1. Die einzelnen Zellen müssen sequentiell, also in Reihe, durchlaufen werden.
2. Die einzelnen Schritte des Codes werden jeder für sich erläutert und dann aufbgebaut, durch Variablen und Funktionen, sodass nachfolgende Zellen von vorherigen Zellen abhängen.
3. Das Notebook kann dabei zwei verschiedene Ziele verfolgen, (i) Durchführung des _grid search_ und mit dem Ziel der Auswahl eines besten Modells, (ii) Auswertung des _topic modeling_ mit dem vorher eruierten, besten Modell. Es ist darauf zu achten, dass der zweite Schritt nur vollzogen werden kann, wenn der erste zumindest einmal durchgeführt wurde, da auf das gespeicherte, beste Modell zugegriffen werden muss.
  3.1 Um das einzustellen, gibt es in Zelle 3 zwei Variablen, die entsprechend zu steuern sind. Alles Weitere passiert dann automatisch.
4. Da insb. der _grid search_ nicht lokal durchgeführt werden konnte, gibt es in der ersten Zelle eine automatische Umgebungserkennung, die registrieren kann, ob man lokal oder in Google Colab arbeitet. In Google Colab wurde mit gekauften Recheneinheiten verfahren und einer A100 gearbeitet.

> Abschließend ist noch anzumerken, dass alle Stellen, die mit der Hilfe moderner KI-Unterstützung überarbeitet wurden, z. B. aufgrund von Fehlerhaftigkeit oder schlechter Funktionalität, als solche gekennzeichnet sind (zum Suchen: "**[KI]**").

## Festlegung der _Pipeline_ (_Grid Search_ oder Modellauswertung)

Im Folgenden muss festgelegt werden, welche Pipeline genommen werden soll. Dafür muss nur die erste Variable eingestellt werden, entweder True oder False.

In [ ]:
##############################################################################
##############################################################################
grid_search_pipeline = False
evaluation_pipeline = True if grid_search_pipeline is False else False

using_all_models = True if grid_search_pipeline is True else False  # Wenn True, werden alle Modelle genutzt, die für das Grid Search gebraucht werden! Embeddings werden neu erstellt und gespeichert!
using_top_models = False if grid_search_pipeline is True else True  # Wenn True, werden nur die beiden besten Modelle genutzt, basierend auf der Grid-Search-Auswertung! Embeddings werden geladen!
##############################################################################
##############################################################################

print(f"Grid Search: {grid_search_pipeline}\nEvaluation: {evaluation_pipeline}\n"
f"Alle Modelle einbeziehen: {using_all_models}\nNur beste Modelle: {using_top_models}")


if grid_search_pipeline is True:
    print("Wenn man den Grid Search durchführen möchte, sollte man sicherstellen, dass man die Hardware dafür hat!")

## Start der Vorbereitungen

In [ ]:
# Diese erste Zeile prüft automatisch, ob in Colab oder lokal gearbeitet wird und setzt die Pfade
# entsprechend zu den unterschiedlichen Verzeichnissen! Außerdem importiert sie die wichtigsten
# Module.

from pathlib import Path
import os
import sys

# Erkennungsvariable als bool setzen
colab_active = "google.colab" in sys.modules

# print(colab_active)

if colab_active is True:
    print("Notebook befindet sich in Colab-Env. Drive wird gemountet und die nötigen"
    " Pakete werden installiert.")

    print("\nDrive wird verbunden.")
    from google.colab import drive
    drive.mount("/content/drive")
    base_dir = Path("/content/drive/Othercomputers/laptop/masterarbeit")

    # cuML GPU Beschleunigung: Import der anderen UMAP und HDBSCAN Module!
    # (Quellen:
    # https://odsc.medium.com/accelerating-umap-processing-10-million-records-in-under-a-minute-with-no-code-changes-5d580deb05a7
    # https://docs.rapids.ai/api/cuml/stable/
    # https://docs.rapids.ai/install/#selector)

    print("\ncuML-Pakete werden importiert.")
    from cuml.manifold import UMAP
    from cuml.cluster import HDBSCAN

    # Installation weiterer Packages für das Colab-Environment
    print("\nInstallation weiterer Pakete.")
    %pip install -q bertopic sentence-transformers hdbscan anthropic --upgrade gensim litellm

    # Pfade werden definiert
    data_dir_pubs = base_dir / "01_data" / "01_csv_data" / "99_pubmed"  # nur PubMed!
    data_dir_tpf = base_dir / "01_data" / "01_csv_data"                 # hier liegen tpf Daten

    embedds_dir = base_dir / "01_data" / "03_topic_modeling" / "01_embeddings"
    topic_results_dir = base_dir / "01_data" / "03_topic_modeling" / "02_topic_results"
    topic_visuals_dir = base_dir / "01_data" / "03_topic_modeling" / "03_topic_visuals"
    grid_search_dir = base_dir / "01_data" / "03_topic_modeling" / "04_grid_search"
    models_dir = base_dir / "01_data" / "03_topic_modeling" / "05_models"

    # Check auf erreichbare Pfade
    all_paths = [base_dir, data_dir_pubs, data_dir_tpf, embedds_dir, topic_results_dir,
                 topic_visuals_dir, grid_search_dir, models_dir]

    no_paths = [x for x in all_paths if not x.exists()]

    if no_paths:
        print(f"Folgende Pfade konnten nicht erreicht werden:\n{no_paths}.")
    else:
        print("\nAlle Pfade konnten gesetzt und gefunden werden!")

else:
    print("Notebook ist lokal! Pfade werden entsprechend der Ordnerstruktur"
          " geprüft und ggf. gesetzt.")

    # Import der nicht-GPU-optimierten Module von UMAP und HDBSCAN
    print("\nImport der UMAP- und HDBSCAN-Module (nicht-GPU-optimiert).")
    from umap import UMAP
    from hdbscan import HDBSCAN

    # Ordnerpfade definieren
    print("\nAufsetzen und Prüfung der Ordnerpfade in lokaler Umgebung.")
    base_dir = Path.cwd().parent # entspricht dem Ordner "masterarbeit"

    # Pfade werden definiert
    data_dir_pubs = base_dir / "01_data" / "01_csv_data" / "99_pubmed"  # nur PubMed!
    data_dir_tpf = base_dir / "01_data" / "01_csv_data"                 # hier liegen tpf Daten

    embedds_dir = base_dir / "01_data" / "03_topic_modeling" / "01_embeddings"
    topic_results_dir = base_dir / "01_data" / "03_topic_modeling" / "02_topic_results"
    topic_visuals_dir = base_dir / "01_data" / "03_topic_modeling" / "03_topic_visuals"
    grid_search_dir = base_dir / "01_data" / "03_topic_modeling" / "04_grid_search"
    models_dir = base_dir / "01_data" / "03_topic_modeling" / "05_models"

    # Check auf erreichbare Pfade
    all_paths = [base_dir, data_dir_pubs, data_dir_tpf, embedds_dir, topic_results_dir,
                 topic_visuals_dir, grid_search_dir, models_dir]

    no_paths = [x for x in all_paths if not x.exists()]

    if no_paths:
        print(f"\nFolgende Pfade konnten nicht erreicht werden:\n{no_paths}.")
    else:
        print("\nAlle Pfade konnten gesetzt und gefunden werden!")

## Alle Importe und Laden der _Transformer Model_

Folgend werden alle notwendigen Importe durchgeführt. Außerdem werden die verschiedenen Transformer-Modelle in einer Dataclass geladen -- je nach Pipeline und Umgebung.

In [ ]:
# from pathlib import Path
from typing import Any, Annotated, Callable, Iterable
import pandas as pd
import re
import openpyxl
import os
import datetime
import time
import traceback
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from dataclasses import dataclass
from dataclasses import asdict
try:
    import kaleido
except Exception as e:
    print("Es gibt fortlaufend Probleme mit Kaleido. Wenn es nicht geladen werden kann,"
    " dann wird es übersprungen. Die Konsequenz ist, dass die Plots nicht als .png "
    "gespeichert werden können.")

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired
import openai
from bertopic.representation import OpenAI
from bertopic.representation import TextGeneration
# from crewai import LLM
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

print("Pakete wurden importiert.")

# NLTK Wordnet Check und Load sowie Setzen der Environ_variables
if not colab_active:
    try:
        nltk_path = Path(r"C:\Users\felix\AppData\Roaming\nltk_data\corpora\wordnet.zip")
        punkt_path = Path(r"C:\Users\felix\AppData\Roaming\nltk_data\tokenizers\punkt_tab")
        stop_words_path = Path(r"C:\Users\felix\AppData\Roaming\nltk_data\corpora\stopwords.zip")

        if nltk_path.exists():
            print("Wordnet-Datei ist schon vorhanden und muss nicht erneut geladen werden!")
        else:
            print("Das NLTK-Paket konnte nicht gefunden werden und wird jetzt geladen!")
            nltk.download("wordnet")

        if punkt_path.exists():
            print("Punkt-Datei ist schon vorhanden und muss nicht erneut geladen werden!")
        else:
            print("Das Punkt-Paket konnte nicht gefunden werden und wird jetzt geladen!")
            nltk.download("punkt_tab")

        if stop_words_path.exists():
            print("Stop-Word-Datei ist schon vorhanden und muss nicht erneut geladen werden!")
        else:
            print("Das Stop-Word-Paket konnte nicht gefunden werden und wird jetzt geladen!")
            nltk.download("stopwords")

        # API KEYs
        try:
            MY_API_KEY = os.environ.get("openaikey1")
            if MY_API_KEY is None:
                print("Der OpenAI-API Key wurde nicht gefunden.")
            else:
                print("Der OpenAI-API Key wurde geladen.")

        except Exception as e:
            print(f"Fehler beim Laden der API Keys: {e}.")

    except Exception as e:
        print(f"Es kam zu einem Fehler: {e}.")

elif colab_active:
    print("Das NLTK- und Punkt-Paket wird geladen.")
    nltk.download("wordnet")
    nltk.download("punkt_tab")
    nltk.download("stopwords")

    # API Key
    MY_API_KEY = None

#########################################################################################################
# Alle wichtigen Vorbereitungsschritte werden hier schon ausgeführt:
#########################################################################################################

# Tagesdatum aktuell festlegen
datum = datetime.datetime.now().strftime("%d.%m.%y")

# Dataclass nutzen, um Sentence-Transformer-Modelle zu organisieren
# (Quelle: https://www.datacamp.com/tutorial/python-data-classes)
@dataclass
class Models:
    raw_instance: SentenceTransformer
    name: str
    trained_instance: object | None = None
    embeddings: np.ndarray | None = None
    doc_topics_assignment: list[int] | None = None
    final_topics_df: pd.DataFrame | None = None
    umap_n_neighbors: int | None = None
    hdbscan_min_cluster_size: int | None = None
    hdbscan_min_samples: int | None = None
    vectorizer_min_df: int | None = None
    vectorizer_max_df: float | None = None

if using_all_models is True and using_top_models is True:
    raise ValueError("\nEs kann nur eine der beiden Optionen auf True gesetzt werden!")
    sys.exit(1)

if colab_active is True:
    if using_all_models is True:

        print("\nColab ist aktiv -- Und: es werden alle Modelle (nicht nur die besten) geladen!")

        # Erstes Dict für die Publikationen
        # model_dict = {
        #     "all-MiniLM": Models(SentenceTransformer("all-MiniLM-L6-v2", device="cuda"), str("all-MiniLM")),
        #     "mpnet-base": Models(SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device="cuda"), str("mpnet-base")),
        #     "pubmed": Models(SentenceTransformer("pritamdeka/S-PubMedBert-MS-MARCO", device="cuda"), str("pubmed")),
        #     "specter": Models(SentenceTransformer("sentence-transformers/allenai-specter", device="cuda"), str("specter"))
        # }

        # Zweites Dict für Drittmittel
        model_dict_tpf = {
            "multilingual-mpnet": Models(SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2", device="cuda"), str("multilingual-mpnet")),
            "multilingual-e5-large": Models(SentenceTransformer("intfloat/multilingual-e5-large", device="cuda"), str("multilingual-e5-large")),
            "LaBSE": Models(SentenceTransformer("sentence-transformers/LaBSE", device="cuda"), str("LaBSE"))
        }

    elif using_top_models is True:

        print("\nColab ist aktiv -- Und: es werden nur die besten Modelle geladen!")

        # Nur die beiden besten Modelle -- diese müssen manuell zugewiesen werden!
        top_model_dict = {
            "LaBSE": Models(SentenceTransformer("sentence-transformers/LaBSE", device="cuda"), str("LaBSE")),
        }

        print(80*"=")

        # Trainierte Modelle laden, falls vorhanden
        if models_dir.exists():
            for x,y in top_model_dict.items():
                print(f"\nStart der Suche nach dem letzten Modellordner für das Modell {y.name}.")
                try:
                    folders = [f.name for f in models_dir.iterdir()]
                    print(f"Liste der Modell-Ordner:\n{folders}")
                    model_folder = [f for f in folders if y.name+"_" in str(f) and "publications" in str(f)]
                    print(f"Ausgewählter Modellordner:\n{model_folder}")

                    if model_folder:
                        latest_folder_name = max(model_folder, key=lambda f: (models_dir / f).stat().st_mtime)
                        latest_model_path = models_dir / latest_folder_name
                        print(f"\nTrainiertes Modell für {y.name} wurde gefunden und wird geladen.")
                        y.trained_instance = BERTopic.load(latest_model_path)
                    else:
                        print(f"\nKein trainiertes Modell im Ordner \"{models_dir}\" für {y.name} gefunden.")

                except Exception as e:
                    print(f"\nFehler beim Finden des neuesten {y}-Modellordners: {e}")

        else:
            print(f"\nDer Modelle-Ordner \"{models_dir}\" konnte nicht gefunden werden.")

        # Document-Topic-Assignment laden, falls vorhanden
        if topic_results_dir.exists():
            for x,y in top_model_dict.items():
                print(f"\nStart der Suche nach der letzten Doc-Topic-Datei für das Modell {y.name}.")
                try:
                    files = [f.name for f in topic_results_dir.glob("*")]
                    print(f"Liste der Doc-Topic-Dateien:\n{files}")
                    file_selection = [f for f in files if y.name+"_" in str(f) and "publications" in str(f)]
                    print(f"Dateiauswahl:\n{file_selection}")

                    if file_selection:
                        latest_file_name = max(file_selection, key=lambda f: (topic_results_dir / f).stat().st_mtime)
                        latest_file_path = topic_results_dir / latest_file_name
                        print(f"\nDocument-Topic-Assignment für {y.name} wurde gefunden und wird geladen.")
                        y.doc_topics_assignment = pd.read_csv(latest_file_path, encoding="utf-8")["topics"].tolist()
                    else:
                        print(f"\nKeine Doc-Topic-Datei im Ordner \"{topic_results_dir}\" für {y.name} gefunden.\n")

                except Exception as e:
                    print(f"\nFehler beim Finden der letzten {y}-Doc-Topic_File: {e}")

        else:
            print(f"\nDie Doc-Topic_Datei in \"{topic_results_dir}\" konnte nicht gefunden werden.")


elif colab_active is False:
    if using_all_models is True:

        print("\nCode läuft lokal -- Und: es werden alle Modelle (nicht nur die besten) geladen!")

        # Erstes Dict für die Publikationen
        # model_dict = {
        #     "all-MiniLM": Models(SentenceTransformer("all-MiniLM-L6-v2"), str("all-MiniLM")),
        #     "mpnet-base": Models(SentenceTransformer("sentence-transformers/all-mpnet-base-v2"), str("mpnet-base")),
        #     "pubmed": Models(SentenceTransformer("pritamdeka/S-PubMedBert-MS-MARCO"), str("pubmed")),
        #     "specter": Models(SentenceTransformer("sentence-transformers/allenai-specter"), str("specter"))
        # }

        # Zweites Dict für Drittmittel
        model_dict_tpf = {
            "multilingual-mpnet": Models(SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2"), str("multilingual-mpnet")),
            "multilingual-e5-small": Models(SentenceTransformer("intfloat/multilingual-e5-small"), str("multilingual-e5-small")),
            "LaBSE": Models(SentenceTransformer("sentence-transformers/LaBSE"), str("LaBSE"))
        }

    elif using_top_models is True:

        print("\nCode läuft lokal -- Und: es werden nur die besten Modelle geladen!\n")

        # Nur die besten Modelle
        top_model_dict = {
            "LaBSE": Models(SentenceTransformer("sentence-transformers/LaBSE"), str("LaBSE"))
        }

        print(80*"=")

        # Trainierte Modelle laden, falls vorhanden
        if models_dir.exists():
            for x,y in top_model_dict.items():
                print(f"\nStart der Suche nach dem letzten Modellordner für das Modell {y.name}.")
                try:
                    folders = [f.name for f in models_dir.iterdir()]
                    print(f"Liste der Modell-Ordner:\n{folders}")
                    model_folder = [f for f in folders if y.name+"_" in str(f) and "drittmittel" in str(f)]
                    print(f"Ausgewählter Modellordner:\n{model_folder}")

                    if model_folder:
                        latest_folder_name = max(model_folder, key=lambda f: (models_dir / f).stat().st_mtime)
                        latest_model_path = models_dir / latest_folder_name
                        print(f"\nTrainiertes Modell für {y.name} wurde gefunden und wird geladen.\n")
                        y.trained_instance = BERTopic.load(latest_model_path)
                    else:
                        print(f"\nKein trainiertes Modell im Ordner \"{models_dir}\" für {y.name} gefunden.\n")

                except Exception as e:
                    print(f"\nFehler beim Finden des neuesten Modellordners für das Modell {y.name}: {e}.\n")

        else:
            print(f"\nDer Modelle-Ordner \"{models_dir}\" konnte nicht gefunden werden.")

        print(80*"=")

        # Document-Topic-Assignment laden, falls vorhanden
        if topic_results_dir.exists():
            for x,y in top_model_dict.items():
                print(f"\nStart der Suche nach der letzten Doc-Topic-Datei für das Modell {y.name}.")
                try:
                    files = [f.name for f in topic_results_dir.glob("*")]
                    print(f"Liste der Doc-Topic-Dateien:\n{files}")
                    file_selection = [f for f in files if y.name+"_" in str(f) and "drittmittel" in str(f) and "doc_topics_assignment_" in str(f)]
                    print(f"Dateiauswahl:\n{file_selection}")

                    if file_selection:
                        latest_file_name = max(file_selection, key=lambda f: (topic_results_dir / f).stat().st_mtime)
                        latest_file_path = topic_results_dir / latest_file_name
                        print(f"\nDocument-Topic-Assignment für {y.name} wurde gefunden und wird geladen.\n")
                        y.doc_topics_assignment = pd.read_csv(latest_file_path, encoding="utf-8")["topics"].tolist()
                    else:
                        print(f"\nKeine Doc-Topic-Datei im Ordner \"{topic_results_dir}\" für {y.name} gefunden.\n")

                except Exception as e:
                    print(f"\nFehler beim Finden der letzten Doc-Topic_File für das Modell {y.name}: {e}.\n")

        else:
            print(f"\nDie Doc-Topic_Datei in \"{topic_results_dir}\" konnte nicht gefunden werden.\n")

        print(80*"=")


## Vorverarbeitung der Daten

Die Publikationsdaten werden eingelesen und vorverarbeitet.

In [ ]:
# Laden der Daten und Vorverarbeitung
df_tpf_raw = pd.read_excel(f"{data_dir_tpf}/raw_data_projects_2003-2025.xlsx",
                   engine="openpyxl")

# Features eingrenzen
df_tpf_raw = df_tpf_raw[["Person: Nachname", "Projekt: Titel des Projekts Deutsch", "Projekt: Titel des Projekts Englisch (GB)",
"Projekt: Stichwörter Deutsch", "Projekt: Stichwörter Englisch (GB)", "Projekt: Projektstart an der Universität Münster",
"Projekt: Projektende an der Universität Münster", "Projekt: Kurzzusammenfassung Englisch (GB)", "Projekt: Kurzzusammenfassung Deutsch", "Projekt: Langbeschreibung Englisch (GB)",
"Projekt: Langbeschreibung Deutsch"]].copy()

# Features umbenennen
df_tpf_raw.rename(columns={"Person: Nachname":"nachname", "Projekt: Titel des Projekts Deutsch":"title_de",
                        "Projekt: Titel des Projekts Englisch (GB)":"titel_en", "Projekt: Stichwörter Deutsch":"keywords_de",
                        "Projekt: Stichwörter Englisch (GB)":"keywords_en", "Projekt: Projektstart an der Universität Münster":"start_date",
                        "Projekt: Projektende an der Universität Münster":"end_date", "Projekt: Kurzzusammenfassung Englisch (GB)":"short_abstract_en",
                        "Projekt: Kurzzusammenfassung Deutsch":"short_abstract_de", "Projekt: Langbeschreibung Englisch (GB)":"long_abstract_en",
                        "Projekt: Langbeschreibung Deutsch":"long_abstract_de"}, inplace=True)

# Check
print(df_tpf_raw.info())

In [ ]:
# Da so viele Daten fehlen und die Einschränkung auf eine Sprache den Datensatz weiter extrem reduzieren würde,
# werden alle verfügbaren String-Daten bilingual zusammengestellt! Der Fokus liegt dabei indes auf den deutschen Angaben,
# die mit den englischen komplementiert werden. Dafür sind auch Sentence-Transformer geladen, die multilingual arbeiten können.

# Zunächst werden alle Duplikate in den Titeln entfernt
df_tpf_processed = df_tpf_raw.drop_duplicates(subset=["title_de", "titel_en"])

# Alle Textdaten in einer Spalte kombinieren
df_tpf_processed["all_combined"] = (
      df_tpf_processed["title_de"].fillna(df_tpf_processed["titel_en"]).fillna("") + " " +            # deutsche Titel werden mit englischen angefüllt
      df_tpf_processed["keywords_de"].fillna(df_tpf_processed["keywords_en"]).fillna("") + " " +      # deutsche Keywords werden mit englischen angefüllt
      df_tpf_processed["long_abstract_de"].fillna(df_tpf_processed["long_abstract_en"]).fillna("")    # deutsche Langbeschreibungen werden mit englischen angefüllt
)

# Wörteranzahl ermitteln
df_tpf_processed["word_count"] = df_tpf_processed["all_combined"].str.split().str.len()

# Kurzer Check
print("Vergleich des ursprünglichen zum vorverarbeiteten Dataframe:\n")
print("\tVerhältnis von neu zu alt = {} %.".format(round((((len(df_tpf_processed) / len(df_tpf_raw))*100)), 0)))
print("\tAnzahl der Wörter insgesamt = {}.".format(df_tpf_processed["word_count"].sum()))
print("\tMedian der Anzahl der Wörter pro Projekt = {}.".format(df_tpf_processed["all_combined"].str.split().str.len().median()))
print(80*"=")
print("\n", "Info ursprünglicher Dataframe:", df_tpf_raw.info())
print(80*"=")
print("\n","Info vorverarbeiteter Dataframe:", df_tpf_processed.info())

### Zwischenfazit zur Datenvorverarbeitung

Es zeigt sich, dass 715 Datensäte erhalten bleiben, wenn man Duplikate entfernt und die Textdaten kombiniert.

Allerdings können nicht alle Projekte auch einem PI zugeordnet werden, insgesamt nur 621. Zudem muss der Zeitrahmen von 2015 bis 2025 eingegrenzt werden.

Darauf erfolgt nun ein weiteres _string preprocessing_, um Sonderzeichen und Abkürzungen zu entfernen.

In [ ]:
# Anzeigeoption einstellen
pd.set_option("display.max_colwidth", 100)

# Check
print(df_tpf_processed[["start_date", "all_combined", "word_count"]].head(50))

In [ ]:
# Regex und Filtern sowie Drop der nicht zugeordneten PI-Projekte

# Regex
def clean_tpf_text(text: str) -> str:
    """Diese Funktion bereinigt den Text von HTML-Tags, Förderkennzeichen und
    sonstigen, unbrauchbaren Textzeichen."""

    if pd.isna(text):
        return ""
    text = re.sub(r"<[^>]+>", "", text)
    text = re.sub(r"\b[A-Z]{2,4}\s+\d+\b", "", text)
    text = re.sub(r"b[A-Z]+-\d{4}-\d+\s*[–-]?\s*", "", text)
    text = re.sub(r"-?\s*\b[A-Z]{2,4}\d+:?\s*", "", text)
    text = re.sub(r":", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df_tpf_processed["all_combined"] = df_tpf_processed["all_combined"].apply(clean_tpf_text)

# Check
print("Check der Regex-Anwendung:\n")
print(df_tpf_processed["all_combined"].head(50))
print(80*"=")

# Filtern und droppen
bool_filter = (df_tpf_processed["start_date"].dt.year >= 2015) & (df_tpf_processed["end_date"].dt.year >= 2015)

df_tpf_processed_filtered = df_tpf_processed[bool_filter]

# Projekte ohne PI droppen
df_tpf_processed_filtered.dropna(subset=["nachname"], inplace=True)

# Check
print("Finaler Check des finalen TPF-DF:\n")
print(df_tpf_processed_filtered.info())
print(80*"=")
print("Relative Anzahl der Projekte nach dem Preprocessing: {} %.".format(round((len(df_tpf_processed_filtered)/len(df_tpf_raw))*100, 0)))
print("Anzahl der verbleibenden Wörter insgesamt = {}.".format(df_tpf_processed_filtered["word_count"].sum()))

### Embeddings werden erstellt

Da die Textvorverarbeitung stattgefunden hat, können jetzt die Embeddings mit den Sentence-Transformern erstellt und gespeichert werden.

In [ ]:
# Die folgende Funktion erstellt ODER lädt bestehende Embeddings der drei verwendeten Modelle
# (Anm.: Da das Erstellen der Embeddings einige Zeit in Anspruch nimmt, wurden diese gespeichert und werden jedes Mal wieder geladen,
# wenn man es so einstellt bzw. sie vorhanden sind. Dafür ist allerdings auch die richtige Ordnerstruktur entscheidend, damit diese
# gespeichert und geladen werden können. )


def create_or_load_embeddings(realm: Annotated[str, "Entweder 'publications' oder 'tpf'"],
                              docs: list[str],
                              model_dict: dict,
                              # model_names: list[str] = [b.name for _,b in model_dict.items()],
                              load_embeds: bool = True) -> np.ndarray:
    """
    Diese Funktion erstellt die Embeddings der Titel+Abstract-Kombinationen für Publikationen und Drittmitteldaten mit vier verschiedenen
    Sentence-Transformer-Modellen:

    1. Allgemeines Modell
    2. PubMed-Modell
    3. SciBERT
    4. SPECTER2

    Dafür werden nur die folgenden Argumente gebraucht:

    Args:
    - realm = "publications" oder "tpf"
    - docs =
    - model_dict =
    - load_embedds =

    Returns:
    - es wird direkt nichts ausgegeben, wohl aber werden die Embeddings in der jeweiligen Model-Class gespeichert!

    """

    ###############################################
    # 1. Teil: Embeddings **laden**
    ###############################################
    # Wenn Arg "load_embeds" == True (Standardwert), dann werden keine Embeddings erstellt, sondern lokal geladen! Es wird automatisch die
    # zuletzt erstelle Datei gesucht

    if load_embeds is True:
        # Hier werden zunächst die jeweils *letzten* Embeddings identifiziert
        embedding_folder_files = list(Path(embedds_dir).glob("*.npy"))

        # Schleife für alle Modelle
        for x,y in model_dict.items():#
            try:
                files = [x for x in embedding_folder_files if y.name + "_" in str(x).lower()]
                latest_file = max(files, key=lambda x: x.stat().st_mtime, default=None)

                model_dict[x].embeddings = np.load(latest_file)

                print(f"Name der zuletzt gespeicherten Embeddings aus dem Modell \"{y.name}\": {latest_file}.")
                print(f"Prüfung der Anzahl der Embeddings von {y.name} und ihrer Dimensionalität:")
                print(f"{y.embeddings.shape[0]} Datensätze mit {y.embeddings.shape[1]} Dimensionen.")
                print(80*"=")

            except Exception as e:
                print(f"Es konnte keine letzte Datei mit Embeddings zu diesem Modell ({y.name}) gefunden werden."
                      f" Es werden jetzt neue Embeddings erstellt. Fehler: {e}.")
                # return False

                try:
                    # Embeddings werden erstellt
                    print(f"Modell \"{y.name}\" startet.")
                    start = time.time()
                    embeddings = y.raw_instance.encode(docs, batch_size=128, show_progress_bar=True)
                    end = time.time()
                    print("Die Embeddings mit {} sind mit einer Laufzeit von {} Minuten erstellt worden.\n".format(y.name, int((end-start)/60)))

                    # Embedings werden gespeichert
                    embeddings_df = pd.DataFrame(embeddings)
                    embeddings_df.to_excel(rf"{embedds_dir}/embeddings_{realm}_{y.name}_{datum}.xlsx",
                                        sheet_name="embeddings", engine="openpyxl")
                    np.save(rf"{embedds_dir}/embeddings_{realm}_{y.name}_{datum}.npy", embeddings)

                    # Embeddings zuweisen
                    y.embeddings = embeddings

                    # Überprüfung
                    print(f"Prüfung der Anzahl der Embeddings von {y.name} und ihrer Dimensionalität:")
                    print(f"{y.embeddings.shape[0]} Datensätze mit {y.embeddings.shape[1]} Dimensionen.")
                    print(80*"=")

                except Exception as e:
                    print(f"Fehler beim Erstellen der Embeddings zum Modell \"{y.name}\": {e}.")

    ###############################################
    # 2. Teil: Embeddings **erzeugen** und speichern (wenn so angegeben!)
    ###############################################
    # Wenn Arg "load_embeds" == False, dann werden keine Embeddings erstellt, sondern geladen.
    # Das Erzeugen der Embeddings dauert je nach Kapazitäten und Modellen bis zu 30 Minuten

    if load_embeds is False:

        # Die Embedding-Objekte sind schon in der ersten Zelle definiert
        for x,y in model_dict.items():

            try:
                # Embeddings werden erstellt
                print(f"Modell \"{y.name}\" startet.")
                start = time.time()
                embeddings = y.raw_instance.encode(docs, batch_size=128, show_progress_bar=True)
                end = time.time()
                print("Die Embeddings mit {} sind mit einer Laufzeit von {} Minuten erstellt worden.\n".format(y.name, int((end-start)/60)))

                # Embedings werden gespeichert
                embeddings_df = pd.DataFrame(embeddings)
                embeddings_df.to_excel(rf"{embedds_dir}/embeddings_{realm}_{y.name}_{datum}.xlsx",
                                    sheet_name="embeddings", engine="openpyxl")
                np.save(rf"{embedds_dir}/embeddings_{realm}_{y.name}_{datum}.npy", embeddings)

                # Embeddings zuweisen
                y.embeddings = embeddings

                # Überprüfung
                print(f"Prüfung der Anzahl der Embeddings von {y.name} und ihrer Dimensionalität:")
                print(f"{y.embeddings.shape[0]} Datensätze mit {y.embeddings.shape[1]} Dimensionen.")
                print(80*"=")

            except Exception as e:
                print(f"Fehler beim Modell {y.name}: {e}.")

In [ ]:
# Es verbleiben 38 % der ursprünglichen Datensätze mit etwa 45.000 Wörtern für das folgende Topic Modeling

# Das Topic Modeling folgt diesem,. auch für die Publikationen angewendeten Aufbau:
# 1. Dataframe laden
# 2. Text-Features erstellen
# 3. Embeddings erstellen und speichern
# 4. Topic-Model trainieren, Topics generieren und beides speichern
# 5. Auswertung der Topics und Speicherung der Ergebnisse

###########################################################################
# DF laden und Text-Feature erstellen -- als Liste
###########################################################################
tpf_cleaned_docs = df_tpf_processed_filtered["all_combined"].tolist()

# print(tpf_cleaned_docs)

###########################################################################
# Embeddings erstellen und speichern mit allen Modellen aus dem Model-Dict-TPF
###########################################################################

if using_all_models is True:    # Dieser Weg führt zum Grid Search, Embeddings werden neu erstellt und gespeichert
    print("Es werden alle Modelle genutzt, für die die Embeddings **neu** erstellt werden.\n")
    create_or_load_embeddings("drittmittel", tpf_cleaned_docs, model_dict_tpf, load_embeds=False)
else:
    print("Embeddings werden hier nicht erstellt, da die Auswertung der besten Modelle folgt.\n")


In [ ]:
# Stopwörter definieren

# Englische Stopwörter aus dem Skript für Publikationen
custom_additions = [
    "the", "and", "of", "in", "to", "is", "for", "was", "we", "that",
    "with", "by", "as", "are", "this", "it", "from", "on", "an", "be",
    "were", "which", "or", "at", "can", "been", "has", "have", "had",
    "they", "their", "these", "those", "than", "then", "them", "there",
    "when", "where", "who", "will", "would", "should", "could", "may",
    "might", "must", "our", "my", "your", "its", "his", "her", "into",
    "through", "during", "before", "after", "above", "below", "up", "down",
    "out", "off", "over", "under", "again", "further", "once", "here",
    "also", "such", "only", "own", "same", "so", "than", "too", "very",
    "can", "just", "don", "now", "use", "using", "used"
]

# Weitere spezifische Stopwörter
academic_stopwords = [
    "study", "studies", "research", "article", "paper", "results", "data",
    "analysis", "methods", "method", "approach", "examined", "investigated",
    "findings", "conclusion", "conclusions", "present", "presented",
    "show", "showed", "shown", "demonstrate", "demonstrated", "found",
    "observed", "reported", "compared", "based", "studied", "analyzed",
    "identified", "examined", "evaluated", "assessed", "determined",
    "associated", "related", "significant", "ignificantly", "effects",
    "effect", "between", "among", "across", "within", "using", "used",
    "experiment", "experiments", "sample", "samples", "population",
    "participants", "subjects", "variables", "variable", "measured",
    "measurement", "results", "conclusions", "implications", "limitations",
    "future", "directions", "introduction", "background", "literature",
    "review", "theory", "theoretical", "framework", "model", "models"
]

# Zusammenführen der Stop-Wörter in einer Liste
comprehensive_stopwords = list(ENGLISH_STOP_WORDS) + custom_additions + academic_stopwords

# Deutsche Stopwörter
standard_german_stops = set(stopwords.words("german"))

scientific_custom_stops = {
    "beziehungsweise", "bzw", "sowie", "daher", "jedoch", "hinsichtlich",
    "bezüglich", "innerhalb", "außerhalb", "aufgrund", "entsprechend",
    "insbesondere", "einschließlich", "beidseitig", "unten", "oben",
    "patient", "patienten", "abbildung", "abb", "tabelle", "tab",
    "ergebnis", "ergebnisse", "methode", "methoden", "studie", "untersuchung",
    "diskussion", "signifikant", "signifikanz", "fall", "falle", "prozent",
    "gruppe", "gruppen", "vergleich", "vs", "et", "al", "ca", "mg", "ml"
}

final_stop_words_ger = standard_german_stops.union(scientific_custom_stops)

comprehensive_stopwords_copy = set(comprehensive_stopwords)

final_final_stop_words = comprehensive_stopwords_copy.union(final_stop_words_ger)

final_stop_words = list(final_final_stop_words)

In [ ]:
# Lemmatization hinzufügen, um die KEywords der Topic Cluster zu verbessern, v.a. um doppelte Wörter zu vermeiden!
# (Quelle: Dokumentation BERTopic unter: https://github.com/MaartenGr/BERTopic/issues/286)
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

###########################################################################################################
# Funktion, um BERTopic zu initialisieren und das Clustering durchzuführen
###########################################################################################################

def topic_clustering(realm: str, sentence_transformer: SentenceTransformer, docs: list[str], embeddings: np.ndarray, hdbscan_min_cluster_size: int = 20, hdbscan_min_samples: int = 5,
                     umap_n_neighbors: int = 15, vec_max_df: float = 0.95, vec_min_df: int = 2, stop_words: list = final_stop_words, save_xlsx: bool = False,
                     model_nr_topics: int | None = None, ai_model: str | None = None, model_name: str | None = None, datum: str | None = None, save_model: bool = False):
    """
    Diese Funktion führt ein Topic Modeling auf Basis von vorbearbeiteten Textdaten sowie bereits erstellen Embeddings durch und nimmt zudem
    eine Liste mit Stop-Wörtern und Hyperparametern zu den Teilmodulen UMAP und HDBSCAN entgegen.

    Args:
    - embedding_model = Hier wird das gewählte Embedding Model übergeben.
    - docs = Hier werden die Datensätze in Form einer Liste übergeben.
    - embeddings = Hier werden die vorkalkulierten Embeddings in einem Array übergeben.
    - hdbscan_min_cluster_size = Hier kann die Anzahl der Topics variiert werden (um die Granularität einzustellen). Standardmäßig ist der in dieser Arbeit beste Wert voreingestellt.
    - umap_n_neighbors = Legt wiederum die Anzahl der Clustergrößen fest
    - model_nr_topics = Hier kann man die Anzahl der Topics im Output direkt festlegen lassen, wenn man bspw. weiß, wie viele es sein sollen.
    - stop_words = Hier wird die erweiterte Stopwörterliste übergeben.

    Returns:
    - Einen Dataframe mit den Ergebnissen des Topic Clusterings.
    - Das trainierte Topic Model.
    """

    # Zeit nehmen
    start_time = time.time()

    try:
        # UMAP zur Dimensionsreduktion mit den Paramtern:
        umap_model = UMAP(n_neighbors=umap_n_neighbors, # legt die Anzahl der Clustergröße fest
                        n_components=5, # legt die Anzahl der Zieldimension fest, auf die reduziert werden soll
                        min_dist=0.0, # legt den Abstand der dimensionreduzierten Clusterpunkte im Raum fest (Unschärfe)
                        metric="cosine", # Metrik zur Bestimmung der Abweichungen/Distanzen
                        random_state=42 # Sorgt für die Reproduzierbarkeit der Ergebnisse
                        )
    except Exception as e:
        print(f"Fehler im UMAP-Algorithmus für das Model {model_name}: {e}.")
        pass

    try:
        # HDBSCAN zum Clustering der Dokumente mit den Parametern
        hdbscan_model = HDBSCAN(min_cluster_size=hdbscan_min_cluster_size, # Minimum an Dokumenten für ein Cluster
                                min_samples=hdbscan_min_samples, # Dichte der Elemente eines Clusters
                                metric="euclidean", # Standardmetrik, passend für wenige Dimensionen
                                cluster_selection_method="eom", # Clusterauswahl; Alternative: leaf
                                prediction_data=True # ermöglicht neue Vorhersagen für neue Dokumente
                            )
    except Exception as e:
        print(f"Fehler im HDBSCAN-Algorithmus für das Model {model_name}: {e}.")
        pass

    try:
        # Vec-Model für stop-word-removal und Erstellung von N-Grammen mit den Parametern:
        vectorizer_model = CountVectorizer(stop_words=stop_words, # Übergabe der definierten Stopwörterliste
                                        tokenizer=LemmaTokenizer(),
                                        min_df=vec_min_df, # wie oft ein Term in den Dokumenten vorkommen **muss**
                                        max_df=vec_max_df, # wie oft ein Term in den Dokumenten vorkommen **darf**
                                        ngram_range=(1, 2), # Angabe der N-Gramme (hier: Ein- bis einschl. Zwei-Wort-Paare)
                                        token_pattern=r"\b[a-zA-Z]{3,}\b" # Akzeptiert werden nur Wörter mit mind. 3 Buchstaben
                                        )
    except Exception as e:
        print(f"Fehler im CountVectorizer für das Model {model_name}: {e}.")
        pass

    # Representation Model (Empfehlung aus der Dokumentation) -- kann man mit OpenAI machen, muss man aber nicht

    # gpt_5_1 = LLM(
    #         model="gpt-5.1",
    #         drop_params=True,
    #         additional_drop_params=["stop"]
    #     )

    representation_model = None
    if ai_model is None:
        print("Es wird kein LLM für die Repräsentationen verwendet!")
        representation_model = KeyBERTInspired()
    elif ai_model == "open_ai":
        print("\nOpenAI soll für das Representation Model genutzt werden.")
        if MY_API_KEY:
            print("\nAPI-Key konnte gefunden werden!")
            client = openai.OpenAI(api_key=MY_API_KEY)

            # Erstellung der Prompts nach Dokumentation von Grootendorst!
            summarization_prompt = """
            I have a topic that is described by the following keywords: [KEYWORDS]
            In this topic, the following documents are a small but representative subset of all documents in the topic:
            [DOCUMENTS]

            Based on the information above, please give a description of this topic in the following format:
            topic: <description>
            """
            title_prompt= """
            I have a topic that contains the following documents:
            [DOCUMENTS]
            The topic is described by the following keywords: [KEYWORDS]

            Based on the information above, extract a short topic label in the following format:
            topic: <topic label>
            """

            #Modeldefinition
            representation_model = {
                "Main": KeyBERTInspired(),
                "ChatGPT": OpenAI(client, model="gpt-4o",
                                  prompt=summarization_prompt,
                                  nr_docs=5,
                                  delay_in_seconds=10 #,
                                  #generator_kwargs={"stop": None}
                                  ),
                "ChatGPT_titles": OpenAI(client, model="gpt-4o",
                                         prompt=title_prompt,
                                         nr_docs=5,
                                         delay_in_seconds=4)
            }
        else:
            print("Kein OpenAI-API-Key vorhanden! KeyBERTInspired wird genommen!")
            representation_model = KeyBERTInspired()

    if representation_model is None:
        raise ValueError("Aufgrund eines Fehlers wurde kein Representation Model geladen!")

    try:
        # BERTopic starten mit den Parametern:
        topic_model = BERTopic(embedding_model=sentence_transformer,        # Sentence-Transformer-Modell
                            umap_model=umap_model,                          # Modell zur Dimensreduktion
                            hdbscan_model=hdbscan_model,                    # Clusteringmodell
                            vectorizer_model=vectorizer_model,              # Vectorizer
                            representation_model=representation_model,      # zwei Representation Models
                            nr_topics=model_nr_topics,                      # Festlegung auf Zielwert der Topic-Anzahl
                            verbose=True,                                   # Fortschrittsanzeige
                            calculate_probabilities=True,                   # Topic-Wahrscheinlichkeiten
                            )
    except Exception as e:
        print(f"Fehler im Aufruf des BERTopic-Moduls für das Modell \"{model_name}\": {e}.")
        pass

    # Beginn des Clusterings der Themen
    topics = None
    probs = None
    try:
        print(f"\nDas Topic Modeling wird jetzt für das Modell \"{model_name}\" initiiert.\n")
        topics, probs = topic_model.fit_transform(docs, embeddings=embeddings)

    except Exception as e:
        if "max_df corresponds to < documents than min_df" in str(e):
            print(80*"=", f"\nBeim Model \"{model_name}\" ist in diesem Durchlauf der bekannte Fehler aufgetreten"
                  f": {e}.\nDie Funktion wird weiter ausgeführt.\n", 80*"=")
        else:
            print(f"Fehler im Topic Modeling für das Modell \"{model_name}\": {e}.")
        return None, None, None, None

    # Outlier autom. reduzieren lassen im Sinne einer nachträglichen Clusterzuordnung
    if topics is not None:
        try:
            new_topics = topic_model.reduce_outliers(docs, topics, strategy="distributions")
            topic_model.update_topics(docs, topics=new_topics, representation_model=representation_model)
            topics = new_topics

            # Speichern
            df_doc_topics = pd.DataFrame({"topics": topics})
            df_doc_topics.to_csv(topic_results_dir / f"doc_topics_assignment_{realm}_{model_name}_{datum}_{hdbscan_min_cluster_size}_{umap_n_neighbors}_{vec_max_df}_{vec_min_df}_{model_nr_topics}_{ai_model}.csv", encoding="utf-8")
            print("\nTopic-Dokumenten-Zuordnung wurde erfolgreich gespeichert.")

        except ValueError as e:
            print(f"Fehler in der Reduzierung der Outlier-Themen beim Modell \"{model_name}\": {e}.")
            new_topics = topics if topics is not None else None
            # Speichern
            df_doc_topics = pd.DataFrame({"topics": new_topics})
            df_doc_topics.to_csv(topic_results_dir / f"doc_topics_assignment_{realm}_{model_name}_{datum}_{hdbscan_min_cluster_size}_{umap_n_neighbors}_{vec_max_df}_{vec_min_df}_{model_nr_topics}_{ai_model}.csv", encoding="utf-8")
            print("Fehler! Die Topic-Dokumenten-Zuordnung wurde dennoch erfolgreich gespeichert.")
    elif topics is None:
            new_topics = None
            print(f"Das Topic Modeling für das Modell \"{model_name}\" konnte nicht durchgeführt werden. Die Topics sind None.")

    # Ergebnisse der Topics als Df einer Variable zuweisen
    topics_summary = topic_model.get_topic_info()

    # Zeit nehmen
    end_time = time.time()
    duration = round(int(((end_time - start_time)/60)), 0)

    # Kurze Auswertungen pro Durchgang ausgeben
    print(80*"=")
    print(f"\nAnalyseergebnisse für das Modell \"{str(model_name)}\" bei einer Laufzeit von {duration if duration > 0 else 1} Minuten:\n")
    print(f"1. Parameter:\n1.1 HDBSCAN min_cluster_size = {hdbscan_min_cluster_size},\n1.1 HDBSCAN min_sample_size ={hdbscan_min_samples},"
        f"\n1.2 UMAP n_neighbors = {umap_n_neighbors},"
        f"\n1.3 Vectorizer max_df = {vec_max_df},"
        f"\n1.4 Vectorizer min_df = {vec_min_df},\n1.5 BERTopic Model min_nr_topics = {model_nr_topics},\n1.6 AI Model = {ai_model}")
    print(f"2. Anzahl der gefundenen Topics = {len(topics_summary)-1}.") # Outlier werden nicht mit angegeben!
    print(f"3. Mittelwert der Publikationen pro Topic = {topics_summary["Count"].mean():.0f}")
    print(f"4. Relation der Outliers am Gesamtkorpus = {((topics_summary["Count"].iloc[0] / topics_summary["Count"].sum())*100):.2f} %.")
    print(f"5. Die Variable \"topics\" enthält die Topic-Zuordnungen pro Dokument und war nicht leer = {topics is not None}.")
    print(80*"=")

    # Ergebnisse pro Durchgang in xlsx und csv lokal speichern
    if save_xlsx is True:
        df = pd.DataFrame(topics_summary)
        df.to_excel(rf"{topic_results_dir}/topic_results_{realm}_{model_name}_{datum}_{hdbscan_min_cluster_size}_{umap_n_neighbors}_{vec_max_df}_{vec_min_df}_{model_nr_topics}_{ai_model}.xlsx", sheet_name="daten", engine="openpyxl")
        df.to_csv(rf"{topic_results_dir}/topic_results_{realm}_{model_name}_{datum}_{hdbscan_min_cluster_size}_{umap_n_neighbors}_{vec_max_df}_{vec_min_df}_{model_nr_topics}_{ai_model}.csv", encoding="utf-8")
        #os.startfile(rf"C:\Users\felix\OneDrive\Desktop\masterarbeit\01_data\03_topic_modeling\topic_results_{model_name}_{datum}_{hdbscan_min_cluster_size}_{umap_n_neighbors}_{vec_max_df}_{vec_min_df}_{model_nr_topics}_{ai_model}.xlsx")

    # Das trainierte Modell ggf. speichern, damit es immer wieder geladen werden kann
    full_path = models_dir / f"{realm}_{model_name}_{datum}_{hdbscan_min_cluster_size}_{umap_n_neighbors}_{vec_max_df}_{vec_min_df}_{model_nr_topics}_{ai_model}"

    # Modell speichern
    if save_model is True:
        try:
            topic_model.save(full_path,
                            serialization="safetensors",
                            save_ctfidf=True,
                            save_embedding_model=sentence_transformer)
        except Exception as e:
            print(f"Fehler beim Speichern des trainierten Modells {model_name}: {e}.")
            pass

    return probs, topics, topics_summary, topic_model

In [ ]:
###########################################################################
# Definition zweier Gensim-Coherence-Score-Funktionen als eines zentralen Gütekriterium für die Topic-Modelle
# (vgl. Röder Röder, Michael, Andreas Both, und Alexander Hinneburg. „Exploring the space of topic coherence measures“.
# Proceedings of the Eighth ACM International Conference on Web Search and Data Mining, 2015)
###########################################################################
# Dieser Code wurde mit Hilfe von Copilot vervollständigt [KI]

from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

def gensim_coherence(trained_topic_model: SentenceTransformer, documents: list[str]) -> float:
    """
    Berechnet den C-V-Score für ein trainiertes BERTopic-Modell.
    """

    topics = trained_topic_model.get_topics()

    topic_words = []
    for topic_id in topics:
        if topic_id != -1:
            words = [word for word, _ in trained_topic_model.get_topic(topic_id)[:10]]
            topic_words.append(words)

    splitted_docs = [doc.lower().split() for doc in documents]

    dict = Dictionary(splitted_docs)

    coherence_model = CoherenceModel(
        topics=topic_words,
        texts=splitted_docs,
        dictionary=dict,
        coherence="c_v"
    )

    score = coherence_model.get_coherence()

    return score

def gensim_coherence_npmi(trained_topic_model: SentenceTransformer, documents: list[str]) -> float:
    """
    Berechnet den C-NPMI-Score für ein trainiertes BERTopic-Modell.
    """

    topics = trained_topic_model.get_topics()

    topic_words = []
    for topic_id in topics:
        if topic_id != -1:
            words = [word for word, _ in trained_topic_model.get_topic(topic_id)[:10]]
            topic_words.append(words)

    splitted_docs = [doc.lower().split() for doc in documents]

    dict = Dictionary(splitted_docs)

    coherence_model = CoherenceModel(
        topics=topic_words,
        texts=splitted_docs,
        dictionary=dict,
        coherence="c_npmi"
    )

    score = coherence_model.get_coherence()

    return score


In [ ]:
###########################################################################
# Topics erstellen
###########################################################################

# Funktion aufrufen, aber mit leicht anderen Baseline-Parametern aufgrund der neuen Daten
# for key, value in model_dict_tpf.items():
#     print(f"Topics werden erstellt mit dem Modell \"{value.name}\".")
#     _, model_dict_tpf[key].doc_topics_assignment, model_dict_tpf[key].final_topics_df, model_dict_tpf[key].trained_instance = topic_clustering(
#         realm="drittmittel",
#         sentence_transformer=model_dict_tpf[key].raw_instance,
#         docs=tpf_cleaned_docs,
#         embeddings=model_dict_tpf[key].embeddings,
#         hdbscan_min_cluster_size=5,
#         hdbscan_min_samples=3,
#         umap_n_neighbors=10,
#         vec_max_df=1.0,
#         vec_min_df=4,
#         stop_words=final_stop_words,
#         model_name=model_dict_tpf[key].name,
#         save_xlsx=True)
#     print(90*"=")

## Hyperparameter-Tuning im Grid Search

In [ ]:
###########################################################################
# Hyperparameter-Tuning
###########################################################################

#########################################################
# Hyperparameter-Tuning-Funktion
#########################################################

def hyperp_tuning(tuple_list: Annotated[tuple, "Hier werden fünf Variablen übergeben: embedding_model, docs, embeddings, model_name, datum"],
                  hdbscan_cluster_range: list[int] = [15],
                  hdbscan_sample_range: list[int] = [5],
                  umap_neighbor_range: list[int] = [10],
                  cv_mindf_range: list[int] = [4],
                  cv_maxdf_range: list[float] = [0.9]) -> pd.DataFrame:
    """
    Diese Funktion ist ein systematisches Hyperparamter-Tuning für die drei ausgewählten Sentence-Transformer und verschiedene
    Hyperparameter in den folgenden Algorithmen: HDBSCAN, UMAP.

    Args:
    -

    Returns:
    - Dataframe mit den Ergebnissen für alle Modelle und Hyperparameter-Ranges
    """

    for a,b,c,d,e in tuple_list:

        for umap in umap_neighbor_range:                    # UMAP n-neighbors alternieren lassen

            for hdbscan_cluster in hdbscan_cluster_range:   # HDBSCAN min_cluster_size alternieren lassen

                for hdbscan_sample in hdbscan_sample_range: # HDBSCAN sample_size alternieren lassen

                    for cv_mindf in cv_mindf_range:         # CountVectorizer alternieren lassen

                        for cv_maxdf in cv_maxdf_range:     # CountVectorizer alternieren lassen

                            probs, topics_all, topics, topic_model = topic_clustering(
                                                                                    realm="drittmittel",
                                                                                    sentence_transformer=a,
                                                                                    docs=b,
                                                                                    embeddings=c,
                                                                                    stop_words=comprehensive_stopwords,
                                                                                    umap_n_neighbors=umap,
                                                                                    hdbscan_min_cluster_size=hdbscan_cluster,
                                                                                    hdbscan_min_samples=hdbscan_sample,
                                                                                    vec_min_df=cv_mindf,
                                                                                    vec_max_df=cv_maxdf,
                                                                                    save_xlsx = True,
                                                                                    save_model=False,
                                                                                    model_name=d,
                                                                                    datum=e)
                            if topics is not None:
                                try:
                                    ergebnisse.append({
                                        "model": str(d),
                                        "umap_n_neighbors": umap,
                                        "hdbscan_min_cluster_size": hdbscan_cluster,
                                        "hdbscan_min_samples_size": hdbscan_sample,
                                        "vectorizer_mind_df": cv_mindf,
                                        "vectorizer_max_df":cv_maxdf,
                                        "count_topics": (len(topics)-1),
                                        "relation_outliers": topics["Count"].iloc[0] / topics["Count"].sum(),
                                        "median_topic_cluster": topics["Count"][1:].median(),
                                        "average_topic_cluster": topics["Count"][1:].mean(),
                                        "topic_cluster_sizes": topics["Count"][1:].tolist(),
                                        "keywords_list": topics["Representation"][1:].tolist(),
                                        "topic_names": topics["Name"][1:].tolist(),
                                        "c_v_score": gensim_coherence(topic_model, b),
                                        "c_npmi_score": gensim_coherence_npmi(topic_model, b)
                                    })
                                except Exception as e:
                                    print(f"Fehler bei Modell {str(d)} während des Speicherns der Ergebnisse: {e}.")
                                    pass
                            else:
                                print("Die Topics sind leer oder fehlerhaft. Die Funktion geht zum nächsten Durchlauf.")

                                ergebnisse.append({
                                        "model": str(d),
                                        "umap_n_neighbors": umap,
                                        "hdbscan_min_cluster_size": hdbscan_cluster,
                                        "hdbscan_min_samples_size": hdbscan_sample,
                                        "vectorizer_mind_df": cv_mindf,
                                        "vectorizer_max_df":cv_maxdf,
                                        "count_topics": 0,
                                        "relation_outliers": 0,
                                        "median_topic_cluster": 0,
                                        "average_topic_cluster": 0,
                                        "topic_cluster_sizes": 0,
                                        "keywords_list": 0,
                                        "topic_names": 0,
                                        "c_v_score": 0,
                                        "c_npmi_score": 0
                                    })

In [ ]:
###########################################################################
# Hyperparameter-Tuning für Drittmitteldaten
###########################################################################

# Erstellung der Ergebnisliste
ergebnisse = []

#############################################################################################
# Funktion aufrufen für Grid-Search
#############################################################################################
if grid_search_pipeline is True:
    print("Es wird das Hyperparameter-Tuning (Grid-Search) für die Drittmitteldaten gestartet.\n")

    tuples_models = []
    for x,y in model_dict_tpf.items():
        tuples_models.append((y.raw_instance, tpf_cleaned_docs, y.embeddings, y.name, datum))

    #############################################################################################
    # Grid-Search durchführen für 3 Modelle mit 5 Parametern und je 3 Werten (3*3*3*3*3*3 = 729 Loops!)
    #############################################################################################

    try:
        start_point = time.time()

        hyperp_tuning(
            tuple_list = tuples_models,
            hdbscan_cluster_range=[10, 15, 20],
            hdbscan_sample_range=[2, 5, 10],
            umap_neighbor_range= [10, 20, 30],
            cv_mindf_range= [2, 3, 4],
            cv_maxdf_range = [0.75, 0.85, 0.90]
            )

    except Exception as e:
        print(f"\nFehler: {e}.")
        traceback.print_exc()

    finally:
        # Ergebnisliste als Dataframe speichern und öffnen
        df_ergebnisse_tpf = pd.DataFrame(ergebnisse)

        df_ergebnisse_tpf.to_excel(rf"{grid_search_dir}/grid_search_results_tpf_{datum}.xlsx",
                            engine="openpyxl", sheet_name="tuning_results")
        df_ergebnisse_tpf.to_csv(rf"{grid_search_dir}/grid_search_results_tpf_{datum}.csv", encoding="utf-8")
        try:
            os.startfile(rf"{grid_search_dir}/grid_search_results_tpf_{datum}.xlsx")
        except Exception as e:
            print("Die Exceldatei konnte nicht geöffnet werden.")

        # Laufzeit final ausgeben
        end_point = time.time()
        dur = round(int((end_point-start_point)/60), 0)
        print(f"Laufzeit des Grid-Search insgesamt {dur} Minuten.")
else:
    print("Das Hyperparameter-Tuning (Grid-Search) für die Drittmitteldaten wird hier nicht durchgeführt.\n")

## Auswertung der Grid-Search-Ergebnisse

In [ ]:
if grid_search_pipeline is True:
    print("Es wird die Ausgabedatei der Grid-Search für die Publikationsdaten eingelesen und ausgewertet.\n")

    # Letzte Grid-Search-Datei finden
    grid_files_tpf = [x for x in grid_search_dir.glob("*") if x.name.startswith("grid_search_results_tpf_")]

    # Nur gültige Excel-Dateien berücksichtigen
    valid_grid_files_tpf = []
    for file in grid_files_tpf:
        try:
            pd.read_excel(file, engine="openpyxl", nrows=1)
            valid_grid_files_tpf.append(file)
        except Exception as e:
            print(f"Warnung: Datei {file.name} konnte nicht gelesen werden ({e}). Sie wird übersprungen.")

    if not valid_grid_files_tpf:
        print("Fehler: Keine gültigen Grid-Search-Dateien gefunden.")
    else:
        latest_grid_file_tpf = max(valid_grid_files_tpf, key=lambda x: x.stat().st_mtime)

        df_grid_search_tpf = pd.read_excel(latest_grid_file_tpf, engine="openpyxl")

        # Checks
        print(f"Spalten des DataFrames: {df_grid_search_tpf.columns.tolist()}")
        print("Länge des Dataframes: ", len(df_grid_search_tpf))

        # Auswertung des Grid-Search in einigen Kennzahlen nach folgenden Überlegungen:
        # 1. Zentral sind die verschiedenen Modelle
        # 2. Die Hyperparameter spielen zunächst keine primäre Rolle, sondern nur die Themenergebnisse
        # 3. count_topics: Anzahl der Topics ist kritisch und ein Gütemaß für das Modell
        # 4. relation_outliers: Ebenfalls kritisch, da das Ausmaß von them. Außenseitern Auskunft über die Breite der Thmene gibt
        # 5. topic_cluster_sizes: Werden genutzt, um zu überprüfen, wie gleichverteilt die Themen sind, was die Dokumente angeht
        # 6. keywords_list und topic_names: Werden genutzt, um zu überprüfen, wie viele wörtliche Überschneidungen es zwischen den Themen gibt

        df_tpf_grouped = df_grid_search_tpf.groupby("model").agg({
            "count_topics":"mean",
            "relation_outliers":"mean",
            #"topic_cluster_sizes":"modus",
            "c_v_score": "mean",
            "average_topic_cluster": ["mean", "min", "max"]
        })

        # Check
        print(df_tpf_grouped)

        # Umwandlung der Keywords-Listen von Strings in Listen-Objekte
        def list_conversion(x):
            """Liest die Strings aus und wandelt sie in verschachtelte Listen um."""

            if pd.isna(x) or x is None:
                return []

            if isinstance(x, str):
                return ast.literal_eval(x)

            return x

        ###############################################
        # Anwendung der Funktion auf die Keyword-Spalte
        ###############################################
        try:
            df_grid_search_tpf["keywords_list"] = df_grid_search_tpf["keywords_list"].apply(list_conversion)
        except Exception as e:
            print(f"Fehler bei der Umwandlung der Keywords-Listen: {e}.")
            pass

        def uniqueness_of_keywords(keywords_list: list[list[str]]) -> float:
            """
            Diese Funktion berechnet den Anteil einzigartiger Keywords in einer Liste von Keyword-Listen.

            Args:
            - keywords_list = Liste von Listen mit Keywords pro Topic

            Returns:
            - Anteil einzigartiger Keywords als Float
            """

            if not isinstance(keywords_list, list):     # Sollte durch Vorverarbeitung mit "list_conversion" ausgeschlossen sein!
                return 0.0

            all_keywords = [x for sublist in keywords_list for x in sublist]
            unique_keywords = set(all_keywords)

            uniqueness_ratio = len(unique_keywords) / len(all_keywords) if all_keywords else 0

            return uniqueness_ratio

        ###############################################
        # Anwendung der Funktion auf die Keywords-Spalte
        ###############################################
        try:
            df_grid_search_tpf["keyword_uniqueness"] = df_grid_search_tpf["keywords_list"].apply(uniqueness_of_keywords)
        except Exception as e:
            print(f"Fehler bei der Berechnung der Keyword-Einzigartigkeit: {e}.")
            pass

        # Finaler Check der Ergebnisse
        try:
            print("Kurzer Check des finalen Df mit den neu erstellten Features.\n")

            # Auswertung des Grid-Search in einigen Kennzahlen nach folgenden Überlegungen:
            # 1. Zentral sind die verschiedenen Modelle
            # 2. Die Hyperparameter spielen zunächst keine primäre Rolle, sondern nur die Themenergebnisse
            # 3. count_topics: Anzahl der Topics ist kritisch und ein Gütemaß für das Modell
            # 4. relation_outliers: Ebenfalls kritisch, da das Ausmaß von them. Außenseitern Auskunft über die Breite der Thmene gibt
            # 5. topic_cluster_sizes: Werden genutzt, um zu überprüfen, wie gleichverteilt die Themen sind, was die Dokumente angeht
            # 6. keywords_list und topic_names: Werden genutzt, um zu überprüfen, wie viele wörtliche Überschneidungen es zwischen den Themen gibt

            df_grouped = df_grid_search_tpf.groupby("model").agg({
                "count_topics":["mean", "min", "max"],
                "relation_outliers":["mean", "min", "max"],
                #"topic_cluster_sizes":"modus",
                "c_v_score": ["mean", "min", "max"],
                "c_npmi_score": ["mean", "min", "max"],
                "average_topic_cluster": ["mean", "min", "max"],
                "keyword_uniqueness": ["mean", "min", "max"]
            })

            print(f"Es folgt eine aggregierte Übersicht der Ergebnisse nach den benutzten Modellen:\n\n{df_grouped}")
            print(80*"=")

        except Exception as e:
            print(f"Fehler bei der Ausgabe der Ergebnisse: {e}.")
            pass

else:
    print("Die Ausgabedatei der Grid-Search für die Publikationsdaten wird hier nicht eingelesen und ausgewertet.\n")

In [ ]:
# Auswertung durchführen mit einer Filterung nach den oben benannten Kriterien in zwei Schritten

if grid_search_pipeline is True:
    print("\nEs wird die Auswertung der Grid-Search-Ergebnisse mit Filterkriterien durchgeführt.\n")

    try:

        df_grid_search_tpf_filtered_strong = df_grid_search_tpf[(df_grid_search_tpf["count_topics"] > 10) &     # Die Anzahl der Themencluster sollte über 13 liegen
                                                (df_grid_search_tpf["keyword_uniqueness"] > 0.80) &    # Die Keywords sollten einen hohen Grad an Einzigartigkeit aufweisen
                                                (df_grid_search_tpf["relation_outliers"] < 0.05) &     # Die Relation der Outlier sollte unter 10 % liegen
                                                (df_grid_search_tpf["c_v_score"] > 0.38)               # Der c_v-Score sollte im oberen Bereich liegen, hier über 0.5
                                                ]

        a1 = len(df_grid_search_tpf_filtered_strong[["model", "count_topics", "relation_outliers", "c_v_score", "keyword_uniqueness", "average_topic_cluster"]])
        print(f"Anzahl der verbleibenden Modelle nach Anwendung der Filterkriterien: {a1}.\n")
        print(df_grid_search_tpf_filtered_strong["model"].value_counts())

        # Zum Gegentesten: Leichte Lockerung der Filterkriterien, um zu sehen, wie sich das auf die Modelle auswirkt
        df_grid_search_tpf_filtered_weak = df_grid_search_tpf[
                                                (df_grid_search_tpf["count_topics"] > 10) &             # Reduziert auf 10 Themen
                                                (df_grid_search_tpf["keyword_uniqueness"] > 0.50) &     # Reduziert auf 0.75
                                                (df_grid_search_tpf["relation_outliers"] < 0.2) &       # Reduziert auf 20 % Outlier
                                                (df_grid_search_tpf["c_v_score"] > 0.30)                # Ein hoher c_v-Score ist relevant!
                                            ]

        print(f"\nModelle mit gelockerten Kriterien: insgesamt {len(df_grid_search_tpf_filtered_weak)} Modelle übrig.\n")
        print(df_grid_search_tpf_filtered_weak["model"].value_counts())

        # # Ranking der verbleibenden Modelle basierend auf dem c_v_score
        print("\nWenn bei einer Lockerung der Filterkriterien viele Modelle übrig bleiben, wird nach dem c_v_score absteigend sortiert,"
            " um zu erkennen, welche Modelle die Top 5 sind:\n")
        df_grid_search_tpf_filtered_weak_sorted = df_grid_search_tpf_filtered_weak.sort_values(by=["c_v_score"], ascending=False)
        print(df_grid_search_tpf_filtered_weak_sorted.head(5))

    except Exception as e:
            print(f"Fehler bei der Filterung der Grid-Search-Ergebnisse: {e}.")
            pass

else:
    print("Die Auswertung der Grid-Search-Ergebnisse mit Filterkriterien wird hier nicht durchgeführt.\n")

In [ ]:
# Die Top-Modelle sind deutlich erkennbar

if grid_search_pipeline is True:
    print("\nDas beste Modell wird jetzt für die finale Analyse gespeichert.\n")

    try:

        top_models_tpf = df_grid_search_tpf_filtered_weak_sorted.iloc[[0]]    # Erstes Modell
        top_models_tpf.to_excel(rf"{grid_search_dir}/top_models_overview_tpf_{datum}.xlsx",
                            engine="openpyxl", sheet_name="top_models_tpf")
        top_models_tpf.to_csv(rf"{grid_search_dir}/top_models_overview_tpf_{datum}.csv",
                              encoding="utf-8")

        print("\nTop-Modell(e) bei den Drittmitteln:")
        print(top_models_tpf)

    except Exception as e:
        print(f"Fehler beim Speichern der Top-Modelle nach Excel: {e}.")
        pass

else:
    print("Das beste Modell für die finale Analyse wird hier nicht gespeichert.\n")

print("\n=========== Stop ===========")
print("An dieser Stelle muss entweder die Evaluationspipeline gestartet werden oder "
"das beste Modell wird neu gewählt (immer daran denken, dass es am Anfang des Notebooks"
" in die Dataclass geladen werden muss!")


### Entscheidung über das beste Modell

An dieser Stelle muss entschieden werden, ob das beste Modell, wie es hier gefiltert wurde, in die Modellklasse geladen werden soll. Dieser Schritt ist deswegen nicht automatisiert.

Dafür muss man in Zelle 3 im top_model_dict das entsprechende Modell eingeben.

In [ ]:
# Check der Werteauslese der besten Modelle

if evaluation_pipeline is True:
    print("\nDie Hyperparameter der/des Top-Modelle/Top-Modells werden jetzt ausgegeben.\n")

    #Finden der zuletzt gespeicherten Datei mit den Top-Modellen
    try:
        top_model_files = [x for x in grid_search_dir.glob("*") if x.name.startswith("top_models_overview_tpf")]
        # print(top_model_files)
        latest_top_model_file = max(top_model_files, key=lambda x: x.stat().st_mtime, default=None)
        # print(latest_top_model_file)

        # csv einlesen
        top_models = pd.read_csv(latest_top_model_file, encoding="utf-8")

        print("\nTop-Modell bei den Drittmitteln:")
        print(top_models.columns)
        print(top_models)

    except Exception as e:
        print(f"Fehler beim finden der der letzten Top-Modelle in Excel: {e}.")
        pass

    try:
        for x in top_models["model"].values:
            aa=int(top_models[top_models["model"] == x]["hdbscan_min_cluster_size"].values[0]),
            bb=int(top_models[top_models["model"] == x]["hdbscan_min_samples_size"].values[0]),
            cc=int(top_models[top_models["model"] == x]["umap_n_neighbors"].values[0]),
            dd=int(top_models[top_models["model"] == x]["vectorizer_mind_df"].values[0]),
            ee=float(top_models[top_models["model"] == x]["vectorizer_max_df"].values[0])
            print(f"\n{x}: HDBSCAN min_cluster_size = {aa},"
            f" HDBSCAN min_sample_size = {bb}, UMAP n_neighbors = {cc},"
            f" Vectorizer min_df = {dd}, Vectorizer max_df = {ee}")
    except Exception as e:
        print(f"Fehler bei der Ausgabe der Hyperparameter der Top-Modelle: {e}.")
        pass

else:
    print("Da die grid-search-pipeline aktiviert ist, wird hier aktuell nicht ausgewertet.")

In [ ]:
#######################################################################
# Prüfung der Top-Modell-Dataclass mit anschließendem Aufruf der Embeddings
#######################################################################

if using_top_models is True:
    for x,y in top_model_dict.items():
        print(f"Die Top-Modell-Dataclass hält aktuell folgende Modelle:\n")
        print(y.name, "\n")

# Suchen und Laden der Embeddings -- falls vorhanden, ansonsten werden diese noch einmal
# erstellt

##########################################
# Funktion zum Check auf lokale Embeddings
##########################################

def check_local_embeddings_list(model_dict: dict) -> list:
    """Diese Funktion checkt, ob für alle Modelle im übergebenen Dict Embeddings
    lokal vorhanden sind.

    Args:
    - model_dict = Dict mit den Modellen

    Returns:
    - list: Gibt eine Liste der Modelle zurück, für die keine Embeddings gefunden werden
    konnten.
    """

    not_exist = []

    for x,y in model_dict.items():
        if y.embeddings is None:
            print(f"Für das Modell {y.name} sind keine lokalen Embeddings vorhanden.")
            not_exist.append(y.name)
        else:
            print(f"Für das Modell {y.name} sind lokale Embeddings vorhanden.")

    return not_exist

# Funktionsaufruf
if using_top_models is True:
    list_no_embedds = check_local_embeddings_list(top_model_dict)

    if not list_no_embedds:
        print("Alle Top-Modelle aus der Data Class haben lokale Embeddings geladen.")
    else:
        print("\nFolgende Modelle haben keine geladenen Embeddings: {}.".format(str(list_no_embedds)))
        print("\nDiese Embeddings werden noch einmal erstellt!")

        # Funktinausfruf zum Erstellen und Speichern der Embeddings oder zum Laden, falls Datei vorhanden
        create_or_load_embeddings("drittmittel", tpf_cleaned_docs, top_model_dict, True) # Wert sollte immer auf True stehen, falls keine Embeddings gefunden werden können, werden sie erstellt

In [ ]:
#######################################################################
# Funktionsuafurf des Topic CLusterings mit Einbindung von ChatGPT im Representation-Model
# Quelle: https://maartengr.github.io/BERTopic/getting_started/representation/llm.html#litellm
# (Anm.: Der Api-Key wird oben schon definiert!)
#######################################################################
# Folgende Schritte werden durchlaufen:
# 1. Modelle werden trainiert und gespeichert -- mit den besten Parametern aus dem Grid-Search und Anthropic als LLM im representation model
# 2. Grafische Auswertung der Topic Clusterings mithilfe der in Bertopic integrierten Visualisierungsfunktionen
# 3. Detaillierte Analyse der Topic Clusterings und deren Keywords

# Funktionsaufruf mit den besten Parametern und den beiden besten Modellen

if evaluation_pipeline is True:

    # Liste zur Speicherung der Ergebnisse der besten Modelle und Entscheidung über die Nutzung von OpenAI-Einbindung
    ergebnisse_best_models = []
    ai_model = "open_ai"   # "open_ai" oder None

    if using_top_models is True:                # Nur, wenn die besten Modelle genutzt werden sollen

        for x, y in top_model_dict.items():

            if y.embeddings is not None:        # Check, ob die Embeddings geladen wurden

                print(f"\nDas Topic Modeling wird jetzt mit dem Modell \"{y.name}\" und ChatGPT als LLM im Representation-Model durchgeführt.")

                try:

                    _, y.doc_topics_assignment, y.final_topics_df, y.trained_instance = topic_clustering(
                                                                                                realm="drittmittel",
                                                                                                sentence_transformer=y.raw_instance,
                                                                                                docs=tpf_cleaned_docs,
                                                                                                embeddings=y.embeddings,
                                                                                                stop_words=comprehensive_stopwords,
                                                                                                hdbscan_min_cluster_size=int(top_models[top_models["model"] == y.name]["hdbscan_min_cluster_size"].values[0]),
                                                                                                hdbscan_min_samples=int(top_models[top_models["model"] == y.name]["hdbscan_min_samples_size"].values[0]),
                                                                                                umap_n_neighbors=int(top_models[top_models["model"] == y.name]["umap_n_neighbors"].values[0]),
                                                                                                vec_min_df=int(top_models[top_models["model"] == y.name]["vectorizer_mind_df"].values[0]),
                                                                                                vec_max_df=float(top_models[top_models["model"] == y.name]["vectorizer_max_df"].values[0]),
                                                                                                save_xlsx=True,
                                                                                                model_name=y.name,
                                                                                                datum=datum,
                                                                                                ai_model=ai_model,
                                                                                                save_model=True)

                except Exception as e:
                    print(f"Fehler beim Topic Modeling mit dem Modell {y.name} und ChatGPT als LLM: {e}.")
                    pass

                if y.final_topics_df is not None:
                    try:
                        ergebnisse_best_models.append({
                            "model": str(y.name),
                            "umap_n_neighbors": int(top_models[top_models["model"] == y.name]["umap_n_neighbors"].values[0]),
                            "hdbscan_min_cluster_size": int(top_models[top_models["model"] == y.name]["hdbscan_min_cluster_size"].values[0]),
                            "hdbscan_min_samples_size": int(top_models[top_models["model"] == y.name]["hdbscan_min_samples_size"].values[0]),
                            "vectorizer_mind_df": int(top_models[top_models["model"] == y.name]["vectorizer_mind_df"].values[0]),
                            "vectorizer_max_df": float(top_models[top_models["model"] == y.name]["vectorizer_max_df"].values[0]),
                            "count_topics": (len(y.final_topics_df)-1),
                            "relation_outliers": y.final_topics_df["Count"].iloc[0] / y.final_topics_df["Count"].sum(),
                            "median_topic_cluster": y.final_topics_df["Count"][1:].median(),
                            "average_topic_cluster": y.final_topics_df["Count"][1:].mean(),
                            "topic_cluster_sizes": y.final_topics_df["Count"][1:].tolist(),
                            "keywords_list": y.final_topics_df["Representation"][1:].tolist(),
                            "topic_names": y.final_topics_df["Name"][1:].tolist(),
                            "topic_names_AI": y.final_topics_df["ChatGPT_titles"][1:].tolist() if ai_model == "open_ai" else None,
                            "topic_summaries_AI": y.final_topics_df["ChatGPT"][1:].tolist() if ai_model == "open_ai" else None,
                            # "topic_names_KeyBERT": y.final_topics_df["Main"][1:].tolist(),
                            "c_v_score": gensim_coherence(y.trained_instance, tpf_cleaned_docs),
                            "c_npmi_score": gensim_coherence_npmi(y.trained_instance, tpf_cleaned_docs)
                        })
                    except Exception as e:
                        print(f"Fehler bei Modell {str(y.name)} während des Speicherns der Ergebnisse: {e}.")
                        pass
                else:
                    print("Die Topics sind leer oder fehlerhaft. Die Funktion geht zum nächsten Durchlauf.")

                    ergebnisse_best_models.append({
                            "model": str(y.name),
                            "umap_n_neighbors": int(top_models[top_models["model"] == y.name]["umap_n_neighbors"].values[0]),
                            "hdbscan_min_cluster_size": int(top_models[top_models["model"] == y.name]["hdbscan_min_cluster_size"].values[0]),
                            "hdbscan_min_samples_size": int(top_models[top_models["model"] == y.name]["hdbscan_min_samples_size"].values[0]),
                            "vectorizer_mind_df": int(top_models[top_models["model"] == y.name]["vectorizer_mind_df"].values[0]),
                            "vectorizer_max_df": float(top_models[top_models["model"] == y.name]["vectorizer_max_df"].values[0]),
                            "count_topics": 0,
                            "relation_outliers": 0,
                            "median_topic_cluster": 0,
                            "average_topic_cluster": 0,
                            "topic_cluster_sizes": 0,
                            "keywords_list": 0,
                            "topic_names": 0,
                            "topic_names_AI": 0,
                            "topic_summaries_AI": 0,
                            # "topic_names_KeyBERT": 0,
                            "c_v_score": 0,
                            "c_npmi_score": 0
                        })

                print(80*"=")

            else:
                print(f"Die Embeddings für das Modell {y.name} wurden nicht geladen.")
                pass

        # Ergebnisse der besten Modelle speichern
        df_ergebnisse_best_models = pd.DataFrame(ergebnisse_best_models)
        df_ergebnisse_best_models.to_excel(rf"{grid_search_dir}/best_models_results_tpf_{datum}.xlsx",
                            engine="openpyxl", sheet_name="best_model_results")
        try:
            os.startfile(rf"{grid_search_dir}/best_models_results_tpf_{datum}.xlsx")
        except Exception as e:
            print(f"Fehler beim Öffnen der Exceldatei! Evtl. OS-Modul vorhanden.")
    else:
        print("Die besten Modelle wurden nicht geladen! BItte erste Zelle prüfen.")

    # Checks
    print(f"Länge der Docs: {len(tpf_cleaned_docs)}.")
    for x,y in top_model_dict.items():
        try:
            print(f"Länge der Topics: {len(y.doc_topics_assignment)}.")
        except Exception as e:
            print(f"Fehler in der Auswertung der Länge: {e}.")

In [ ]:
# Check auf korrekte befüllte Modellklassen

if evaluation_pipeline is True:
    print("\nEs wird ein Check der befüllten Modellklassen der Top-Modelle durchgeführt.\n")

    try:
        for x, y in top_model_dict.items():
            print(f"\nModell: {y.name}")
            print(f"Document-Topic-Zuweisungen: {type(y.doc_topics_assignment)}")
            print(f"Länge der Document-Topic-Zuweisung: {len(y.doc_topics_assignment)}")
            print(f"Finales Topic-Df: {type(y.final_topics_df)}")
            print(f"Länge des finalen Topic-Df: {len(y.final_topics_df)-1}") # Outlier müssen herausgenommen werden!
            print(f"Trainiertes Modell: {type(y.trained_instance)}")
    except Exception as e:
        print(f"Fehler beim Check der Modellklassen: {e}.")
        pass

In [ ]:
###########################################################
# Funktion definieren für insg. sieben Visualisierungen
###########################################################

def visuals_per_topic_model(
    realm: str,
    topic_model: Annotated[object, "Hier muss das trainierte topic.model übergeben werden"],
    docs: list[str],
    top_n_topics: int = 10,
    n_words: int = 10,
    model_name: str | None = None,
    embeddings: np.ndarray | None = None,
    sample_documents: list | None = None,
    visualizations: Annotated[list[str], "Eine Auswahl aus heatmap, topics, barchart, term_rank, hierarchy, documents, hierarchical_documents]"] = None,
    show_visuals: bool = False
) -> dict:

    """
    Sammelfunktion, um die graphischen Auswertungen zu den Topic-Clustern zu erstellen und lokal zu speichern!

    Args:
    - ...

    Returns:
    - Dict der erstellen Plots
    """

    # Ordner Set Up mit Pathlib
    current_dir = Path.cwd()
    target_folder = current_dir.parent / "01_data" / "03_topic_modeling"
    topic_visuals_folder = target_folder / "03_topic_visuals"

    # Zielordner checken (relativ zum Notebook)
    try:
        if Path(topic_visuals_folder).exists():
            print(f"Ordner \"{str(topic_visuals_folder)}\" vorhanden.")
        else:
            print(f"Ordner \"{str(topic_visuals_folder)}\" nicht vorhanden. Der Ordner wird im folgenden Schritt erstellt werden.")
            topic_visuals_folder.mkdir(parents=True)
    except Exception as e:
        print(f"Aufgrund eines Fehlers konnte nicht weiter verfahren werden: {e}.")
    print(80*"=")

    # Dict initialisieren, um die Visualisierungen zu speichern!
    figures = {}

    if visualizations is None:
        visualizations = ["heatmap", "topics", "barchart", "term_rank",
                         "hierarchy", "documents", "hierarchical_documents"]

    ###########################################################################
    # 1. Topic Heatmap
    ###########################################################################
    if "heatmap" in visualizations:
        try:
            print("1. Topic Heatmap erstellen.")

            # Plot erstellen!
            fig = topic_model.visualize_heatmap()
            # Plot dem Dict zuweisen
            figures["heatmap"] = fig

            # Ergebnis speichern
            path = rf"{topic_visuals_folder}/heatmap_{realm}_{model_name}_{datum}"
            fig.write_html(rf"{path}.html")
            try:
                fig.write_image(rf"{path}.png")
            except Exception as e:
                print(f"PNG-Export fehlgeschlagen (kaleido Version ist vorhanden!): {e}")
            # Plot ausgeben lassen
            if show_visuals is True:
                fig.show()

            print(f"Plot gespeichert: \"{path}\"")
        except Exception as e:
            print(f"Fehler \"{e}\".")

    ###########################################################################
    # 2. 2D Topic Map
    ###########################################################################
    if "topics" in visualizations:
        try:
            print("2. Topic Map erstellen.")

            # Plot erstellen
            fig = topic_model.visualize_topics(width=1200, height=800)
            # Plot dem Dict zuweisen
            figures["topics"] = fig

            # Ergebnis speichern
            path = rf"{topic_visuals_folder}/topic-map_{realm}_{model_name}_{datum}.html"
            fig.write_html(path)
            try:
                fig.write_image(path.replace(".html", ".png"))
            except Exception as e:
                print(f"PNG-Export fehlgeschlagen (kaleido 0.2.1 ist aber vorhanden!): {e}")

            # Plot anzeigen lassen
            if show_visuals is True:
                fig.show()

            print(f"Plot gespeichert: \"{path}\"")
        except Exception as e:
            print(f"Fehler \"{e}\".")

    ###########################################################################
    # 3. Topic Words Bar Chart
    ###########################################################################
    if "barchart" in visualizations:
        try:
            print("3. Topic Barchart erstellen.")

            # Plot erstellen
            fig = topic_model.visualize_barchart(
                top_n_topics=top_n_topics,
                n_words=n_words,
                height=500
            )
            # Plot dem Dict zuweisen
            figures["barchart"] = fig

            # Ergebnis speichern
            path = rf"{topic_visuals_folder}/bar-chart-words_{realm}_{model_name}_{datum}.html"
            fig.write_html(path)
            try:
                fig.write_image(path.replace(".html", ".png"))
            except Exception as e:
                print(f"PNG-Export fehlgeschlagen (kaleido 0.2.1 ist aber vorhanden!): {e}")

            # Plot anzeigen lassen
            if show_visuals is True:
                fig.show()

            print(f"Plot gespeichert: \"{path}\"")
        except Exception as e:
            print(f"Fehler \"{e}\".")

    ###########################################################################
    # 4. Term Rank Plot
    ###########################################################################
    if "term_rank" in visualizations:
        try:
            print("4. Term Ranking erstellen")

            # Plot erstellen
            fig = topic_model.visualize_term_rank(log_scale=True)
            # Plot dem Dict zuweisen
            figures["term_rank"] = fig

            # Plot speichern und anzeigen lassen
            path = rf"{topic_visuals_folder}/term-rank_{realm}_{model_name}_{datum}.html"
            fig.write_html(path)
            try:
                fig.write_image(path.replace(".html", ".png"))
            except Exception as e:
                print(f"PNG-Export fehlgeschlagen (kaleido 0.2.1 ist aber vorhanden!): {e}")

            # Plot anzeigen lassen
            if show_visuals is True:
                fig.show()

            print(f"Plot gespeichert: \"{path}\"")
        except Exception as e:
            print(f"Fehler \"{e}\".")

    ###########################################################################
    # 5. Hierarchical Topic Structure
    ###########################################################################
    if "hierarchy" in visualizations:
        try:
            print("5. Topic Hierarchy erstellen.")

            # Hierarchie erzeugen
            hierarchical_topics = topic_model.hierarchical_topics(docs)

            # Plot erstellen lassen
            fig = topic_model.visualize_hierarchy(
                hierarchical_topics=hierarchical_topics,
                width=1200,
                height=800
            )

            # Plot dem Dict zuweisen
            figures["hierarchy"] = fig

            path = rf"{topic_visuals_folder}/hierarchy_{realm}_{model_name}_{datum}.html"
            fig.write_html(path)
            try:
                fig.write_image(path.replace(".html", ".png"))
            except Exception as e:
                print(f"PNG-Export fehlgeschlagen (kaleido 0.2.1 ist aber vorhanden!): {e}")

            # Plot anzeigen lassen
            if show_visuals is True:
                fig.show()

            print(f"Gespeichert: \"{path}\".")

            # Speichern:
            tree = topic_model.get_topic_tree(hierarchical_topics)
            tree_path = rf"{topic_visuals_folder}/topic-tree_{realm}_{model_name}_{datum}.html"
            with open(tree_path, "w", encoding="utf-8") as f:
                f.write(tree)
                try:
                    fig.write_image(path.replace(".html", ".png"))
                except Exception as e:
                    print(f"PNG-Export fehlgeschlagen (kaleido 0.2.1 ist aber vorhanden!): {e}")
            print(f"Gespeichert: \"{tree_path}\".")

        except Exception as e:
            print(f"Fehler: {e}")

    ###########################################################################
    # 6. Document Distribution
    ###########################################################################
    if "documents" in visualizations:
        if embeddings is not None:
            try:
                print("6. Dokumenten-Karte erstellen.")

                # Plot erstellen
                fig = topic_model.visualize_documents(
                    docs,
                    embeddings=embeddings,
                    sample=sample_documents,
                    width=1500,
                    height=1000
                )

                # Plot dem Dict zuweisen
                figures["documents"] = fig

                path = rf"{topic_visuals_folder}/doc-map-2D_{realm}_{model_name}_{datum}.html"
                fig.write_html(path)
                try:
                    fig.write_image(path.replace(".html", ".png"))
                except Exception as e:
                    print(f"PNG-Export fehlgeschlagen (kaleido 0.2.1 ist aber vorhanden!): {e}")

                # Plot anzeigen lassen
                if show_visuals is True:
                    fig.show()

                print(f"Gespeichert: \"{path}\".")

            except Exception as e:
                print(f"Fehler: {e}")
        else:
            print("6. Da keine Embeddings übergeben wurden, konnte nicht geplottet werden.")

    ###########################################################################
    # 7. Hierarchical Document Visualization
    ###########################################################################
    if "hierarchical_documents" in visualizations:
        if embeddings is not None:
            try:
                print("7. Dokumenten-Karte in 2D nach Hierarchie erstellen.")

                # Check, ob die Hierarchie schon erstellt wurde
                if "hierarchy" not in figures:
                    hierarchical_topics = topic_model.hierarchical_topics(docs)

                # Plot erstellen
                fig = topic_model.visualize_hierarchical_documents(
                    docs,
                    hierarchical_topics=hierarchical_topics,
                    embeddings=embeddings,
                    sample=sample_documents,
                    width=1500,
                    height=1000
                )

                # Plot dem Dict zuweisen
                figures["hierarchical_documents"] = fig

                # Plotten und anzeigen lassen
                path = rf"{topic_visuals_folder}/hier-doc-map-2D_{realm}_{model_name}_{datum}.html"
                fig.write_html(path)
                try:
                    fig.write_image(path.replace(".html", ".png"))
                except Exception as e:
                    print(f"PNG-Export fehlgeschlagen (kaleido >=1.0.0 ist aber vorhanden!): {e}")

                # Plot anzeigen lassen
                if show_visuals is True:
                    fig.show()

                # Speichern
                print(f"Gespeichert: \"{path}\".")

            except Exception as e:
                print(f"Fehler: \"{e}\".")

        else:
            print("7. Da keine Embeddings übergeben wurden, konnte nicht geplottet werden.")

    print(f"\nAuswertung: Es wurden insgesamt {len(figures)} Plots erstellen und in folgendem Ordner gespeichert: {topic_visuals_folder}.\n")

    return figures


In [ ]:
###########################################################################
# Auswertung der Ergebnisse in Plots und Graphiken
###########################################################################

if evaluation_pipeline is True:
    print("\nDie Visualisierungen für die Top-Modelle werden jetzt erstellt und gespeichert.\n")

    figures_dict = {}

    for x,y in top_model_dict.items():

        figures_pubs = visuals_per_topic_model(
                                                realm="drittmittel",
                                                topic_model=y.trained_instance,
                                                docs=tpf_cleaned_docs,
                                                model_name=y.name,
                                                embeddings=y.embeddings,
                                                show_visuals=True
                                            )

        figures_dict[y.name] = figures_pubs

else:
    print("Da die grid-search-pipeline aktiv ist, wird hier nicht ausgewertet.")

## Dynamisches Topic Modeling

In [ ]:
print(df_tpf_processed_filtered["start_date"].head())

In [ ]:
###########################################################
# Dynamisches Topic Modeling nach Jahren
# Quelle: https://maartengr.github.io/BERTopic/getting_started/topicsovertime/topicsovertime.html#bins
###########################################################

# Datenvorvereitung mit den Jahresdaten mit dem ursprünglichen Dataframe
if evaluation_pipeline is True:
    print("\nDie Jahresdaten für das dynamische Topic Modeling werden jetzt vorbereitet.\n")

    date_list = df_tpf_processed_filtered["start_date"].dt.year.tolist()

    df_aggre = df_tpf_processed_filtered.groupby("start_date")["title_de"].count()
    print(df_aggre)

    # Check der Längen
    print(f"Dokumente zu Datumsangaben passend = {len(date_list) == len(tpf_cleaned_docs)}.")

    print(f"Länge der Jahresangaben = {len(date_list)}.")
    print(f"Jahreszeitraum: {min(date_list)} - {max(date_list)}")
    print(f"Verteilung der Jahresdaten:\n{sorted(set(date_list))}")

In [ ]:
# Aufruf der BERTopic-Funktion für dynamisches Topic Modeling mit den besten Modellen
# (Anm.: Die Berechnung dauert bis zu 25 Minuten für zwei Graphen!)

if evaluation_pipeline is True:
    print("\nDas dynamische Topic Modeling für die Top-Modelle wird jetzt durchgeführt und die Graphen werden ausgegeben.\n")

    for x, y in top_model_dict.items():
        topics_over_time = y.trained_instance.topics_over_time(
                                                        docs=tpf_cleaned_docs,
                                                        timestamps=date_list,
                                                        #datetime_format="%Y",
                                                        nr_bins=len(set(date_list))
                                                        )

        fig = y.trained_instance.visualize_topics_over_time(topics_over_time)
        fig.write_html(rf"{topic_visuals_dir}/dynamic_topic_modeling_{y.name}_{datum}.html")
        try:
            fig.write_image(rf"{topic_visuals_dir}/dynamic_topic_modeling_{y.name}_{datum}.png")
            fig.show()
        except Exception as e:
            print(f"PNG-Export fehlgeschlagen (kaleido 0.2.1 ist aber vorhanden!): {e}")

## Rückschreibung der Ergebnisse an die PIs

In [ ]:
##########################################################################################
# Drittmittel
##########################################################################################

if evaluation_pipeline is True:
    print("\nDie erstellten Topics der Top-Modelle werden jetzt den Drittmitteln zugeordnet.\n")

    # 2.1 Themencluster für Drittmittel zuordnen in den ursprünglichen Dataframe
    print(df_tpf_processed_filtered.columns)
    print(f"Länge des Dataframes: {len(df_tpf_processed_filtered)}.")
    print(f"Länge der Dokumente: {len(tpf_cleaned_docs)}.")

    for x,y in top_model_dict.items():
        print(f"\nDokument-Topic-Zuweisungen für Modell \"{y.name}\" werden dem Dataframe hinzugefügt.")
        df_tpf_processed_filtered[f"topic_assignment_{y.name}"] = y.doc_topics_assignment

    # 2.2 Themen und Keywords zuordnen in den ursprünglichen Dataframe
    for x,y in top_model_dict.items():

        y.final_topics_df = y.trained_instance.get_topic_info()

        print(f"\nThemen und Keywords für Modell \"{y.name}\" werden dem Dataframe hinzugefügt.")
        topic_names = y.final_topics_df.set_index("Topic")["Name"].to_dict()
        topic_keywords = y.final_topics_df.set_index("Topic")["Representation"].to_dict()

        df_tpf_processed_filtered[f"topic_name_{y.name}"] = df_tpf_processed_filtered[f"topic_assignment_{y.name}"].map(topic_names)
        df_tpf_processed_filtered[f"topic_keywords_{y.name}"] = df_tpf_processed_filtered[f"topic_assignment_{y.name}"].map(topic_keywords)

    print(f"\nNeue Spalten Drittmittel: {df_tpf_processed_filtered.columns}.")
    df_tpf_processed_filtered.head()

else:
    print("Die erstellten Topics der Top-Modelle werden hier nicht den Drittmitteln zugeordnet. Dafür muss die "
          "evaluation_pipeline Variable auf True gesetzt werden.\n")


In [ ]:
##########################################################################################
# Drittmittel
##########################################################################################

if evaluation_pipeline is True:
    print("\nDie aggregierten Metriken für PIs aus Drittmitteln werden jetzt erstellt.\n")

    for x,y in top_model_dict.items():
        print(f"\nAggregierte Metriken für PIs aus Drittmitteln werden für Modell \"{y.name}\" erstellt.")

        # Anonymisierung der PI-Namen im Dataframe
        pi_df = pd.read_csv(rf"{base_dir}/01_data/01_csv_data/00_pi_basics/FINALLY_ALL_pi_data.csv", encoding="utf-8")
        pi_hash_dict = pi_df.set_index("nachname")["pi_name_hashed"].to_dict()

        df_tpf_processed_filtered["nachname"] = df_tpf_processed_filtered["nachname"].map(pi_hash_dict)

        pi_metrics_tpf = df_tpf_processed_filtered.groupby("nachname").agg(
            topic_count=(f"topic_assignment_{y.name}", "nunique"),
            total_tpfs=(f"topic_assignment_{y.name}", "size")
        ).reset_index()

        # Top 3 topics
        pi_metrics_tpf["top3_topics_nr"] = df_tpf_processed_filtered.groupby("nachname")[f"topic_assignment_{y.name}"].apply(
            lambda x: x.value_counts().head(3).index.tolist()).values
        pi_metrics_tpf["top3_topics_title"] = df_tpf_processed_filtered.groupby("nachname")[f"topic_name_{y.name}"].apply(
            lambda x: [x for x in x.value_counts().head(3).index.tolist()]).values

        # Speichern als csv
        pi_metrics_tpf.to_csv(rf"{topic_results_dir}/pi_tpf_metrics_{y.name}_{datum}.csv", index=False, encoding="utf-8")
        pi_metrics_tpf.to_excel(rf"{topic_results_dir}/pi_tpf_metrics_{y.name}_{datum}.xlsx",
                            engine="openpyxl", sheet_name="pi_metrics")

    # Check
    print(pi_metrics_tpf)

else:
    print("Die aggregierten Metriken für PIs aus Drittmitteln werden hier nicht erstellt. Dafür muss die "
            "evaluation_pipeline Variable auf True gesetzt werden.\n")